In [2]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import time
import datetime as dt

df = pd.read_csv("data.csv")
df.head()

,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,...,usd_type,creator,location,category,profile,spotlight,urls,percent_funded,is_liked,is_disliked
0,854926212,{'key': 'assets/031/658/577/009bbf11b7764224ee...,khōréō: a new magazine of speculative fiction,A quarterly magazine of speculative fiction by...,7000.0,7747.00,live,khoreo-a-new-magazine-of-speculative-fiction,False,US,...,international,"{'id': 1125956615, 'name': 'khōréō magazine', ...","{'id': 2459115, 'name': 'New York', 'slug': 'n...","{'id': 326, 'name': 'Literary Journals', 'slug...","{'id': 3999404, 'project_id': 3999404, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,110.671429,False,False
1,499055298,{'key': 'assets/031/934/565/2da00cd9c7477ec667...,Mostly Monsters: The Art of Allison Theus,150+ pages of art by concept artist Allison Th...,40000.0,47757.00,live,mostly-monsters-the-art-of-allison-theus,False,US,...,international,"{'id': 1260269319, 'name': 'Allison Theus', 'i...","{'id': 2357536, 'name': 'Austin', 'slug': 'aus...","{'id': 22, 'name': 'Illustration', 'slug': 'ar...","{'id': 4101278, 'project_id': 4101278, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,119.392500,False,False
2,775337554,{'key': 'assets/031/961/688/b4c793731912a3cb5b...,The Bradley Cinema,The Bradley will be a 3-screen 1st run movie t...,50000.0,3311.00,live,the-bradley-cinema,False,US,...,international,"{'id': 1186616842, 'name': 'CINEMA LAB', 'slug...","{'id': 2356439, 'name': 'Asbury Park', 'slug':...","{'id': 298, 'name': 'Movie Theaters', 'slug': ...","{'id': 4137276, 'project_id': 4137276, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,6.622000,False,False
3,733850575,{'key': 'assets/031/979/188/85438460bd8e77e4a2...,Darwin's Journey,Retracing Darwin’s journey to the Galapagos in...,30000.0,518608.50,live,darwins-journey,False,ES,...,international,"{'id': 1987827713, 'name': 'Gonzalo Aguirre Bi...","{'id': 12689383, 'name': 'Sevilla', 'slug': 's...","{'id': 34, 'name': 'Tabletop Games', 'slug': '...","{'id': 4115909, 'project_id': 4115909, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,1728.695000,False,False
4,242291145,{'key': 'assets/031/697/886/a4728e817030ad8a29...,Ett hjärta är alltid rött - A heart is always red,En dokumentärfilm om bandet IMPERIET - \nA doc...,200000.0,530833.71,live,ett-hjarta-ar-alltid-rott,False,SE,...,international,"{'id': 1117518154, 'name': 'DOCSTER', 'slug': ...","{'id': 906057, 'name': 'Stockholm', 'slug': 's...","{'id': 30, 'name': 'Documentary', 'slug': 'fil...","{'id': 3862455, 'project_id': 3862455, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,265.416855,False,False


In [3]:
len(df)

2400

In [4]:
df.columns

Index(['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug',
       'disable_communication', 'country', 'country_displayable_name',
       'currency', 'currency_symbol', 'currency_trailing_code', 'deadline',
       'state_changed_at', 'created_at', 'launched_at', 'staff_pick',
       'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged',
       'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type',
       'creator', 'location', 'category', 'profile', 'spotlight', 'urls',
       'percent_funded', 'is_liked', 'is_disliked'],
      dtype='object')

In [6]:
print(f"Of the {len(df)} projects in the dataset, there are {len(df[df.duplicated(subset='id')])} which are listed more than once.")
df = df.drop_duplicates(subset='id', keep="first")
print(f"\nOf the {len(df)} projects in the dataset, there are {len(df[df.duplicated(subset='id')])} which are listed more than once.")

Of the 2400 projects in the dataset, there are 2 which are listed more than once.

Of the 2398 projects in the dataset, there are 0 which are listed more than once.


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2398 entries, 0 to 2399
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        2398 non-null   int64  
 1   photo                     2398 non-null   object 
 2   name                      2398 non-null   object 
 3   blurb                     2398 non-null   object 
 4   goal                      2398 non-null   float64
 5   pledged                   2398 non-null   float64
 6   state                     2398 non-null   object 
 7   slug                      2398 non-null   object 
 8   disable_communication     2398 non-null   bool   
 9   country                   2398 non-null   object 
 10  country_displayable_name  2398 non-null   object 
 11  currency                  2398 non-null   object 
 12  currency_symbol           2398 non-null   object 
 13  currency_trailing_code    2398 non-null   bool   
 14  deadline

In [9]:
# Dropping columns that aren't useful
df.drop(['converted_pledged_amount', 'creator', 'currency', 'currency_symbol', 'currency_trailing_code', 'current_currency', 'fx_rate', 'photo', 'pledged', 'profile', 'slug', 'spotlight', 'state_changed_at', 'urls', 'usd_type'], axis=1, inplace=True)

In [10]:
# Converting dates from unix to datetime
cols_to_convert = ['created_at', 'deadline', 'launched_at']
for c in cols_to_convert:
    df[c] = pd.to_datetime(df[c], origin='unix', unit='s')

In [11]:
print(f"The dataset contains projects added to Kickstarter between {min(df.created_at).strftime('%d %B %Y')} and {max(df.created_at).strftime('%d %B %Y')}.")

The dataset contains projects added to Kickstarter between 08 February 2015 and 12 January 2021.


In [12]:
df.dtypes

id                                   int64
name                                object
blurb                               object
goal                               float64
state                               object
disable_communication                 bool
country                             object
country_displayable_name            object
deadline                    datetime64[ns]
created_at                  datetime64[ns]
launched_at                 datetime64[ns]
staff_pick                            bool
is_starrable                          bool
backers_count                        int64
static_usd_rate                    float64
usd_pledged                        float64
location                            object
category                            object
percent_funded                     float64
is_liked                              bool
is_disliked                           bool
dtype: object